1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰 수 초과로 답변을 생성하지 못할 수 있음
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 백터 데이터베이스에 저장
4. 질문이 있을 때, 백터 데이터베이스에서 유사도 검사
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [26]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # LLM은 토큰 수를 계산, 청크사이즈는 문서를 쪼갤 때 하나의 청크가 가질 수 있는 토큰 수
    # 오버랩은 겹치는 토큰 수, 오버랩을 주면서 유사도를 높일 수 있음
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [27]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [28]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import time

index_name = 'tax-index'
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pinecone_api_key



pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [41]:
query = '연봉 5천만원인 거주자의 종합소득세는 얼마인가요?'

In [42]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = 'gpt-4o')

In [43]:
from langchain import hub

prompt = hub.pull('rlm/rag-prompt')

In [44]:
retriever = database.as_retriever(search_kwargs={'k':4})
retriever.invoke(query)

[Document(metadata={'source': './tax.docx'}, page_content='3. 「자본시장과 금융투자업에 관한 법률」 제251조제1항에 따른 집합투자업겸영보험회사의 특별계정\n\n③ 비거주자의 소득은 제119조에 따라 구분한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제4조(소득의 구분) ① 거주자의 소득은 다음 각 호와 같이 구분한다. <개정 2013. 1. 1., 2020. 12. 29.>\n\n1. 종합소득\n\n\u3000이 법에 따라 과세되는 모든 소득에서 제2호, 제2호의2 및 제3호에 따른 소득을 제외한 소득으로서 다음 각 목의 소득을 합산한 것\n\n가. 이자소득\n\n나. 배당소득\n\n다. 사업소득\n\n라. 근로소득\n\n마. 연금소득\n\n바. 기타소득\n\n2. 퇴직소득\n\n2의2. 금융투자소득\n\n3. 양도소득\n\n② 제1항에 따른 소득을 구분할 때 다음 각 호의 신탁을 제외한 신탁의 이익은 「신탁법」 제2조에 따라 수탁자에게 이전되거나 그 밖에 처분된 재산권에서 발생하는 소득의 내용별로 구분한다.<개정 2011. 7. 25., 2020. 12. 29., 2022. 12. 31.>\n\n1. 「법인세법」 제5조제2항에 따라 신탁재산에 귀속되는 소득에 대하여 그 신탁의 수탁자가 법인세를 납부하는 신탁\n\n2. 「자본시장과 금융투자업에 관한 법률」 제9조제18항제1호에 따른 투자신탁. 다만, 2024년 12월 31일까지는 이 법 제17조제1항제5호에 따른 집합투자기구로 한정한다.\n\n3. 「자본시장과 금융투자업에 관한 법률」 제251조제1항에 따른 집합투자업겸영보험회사의 특별계정\n\n③ 비거주자의 소득은 제119조에 따라 구분한다.\n\n[전문개정 2009. 12. 31.]\n\n[시행일: 2025. 1. 1.] 제4조제1항제1호, 제4조제1항제2호의2\n\n\n\n제5조(과세기간) ① 소득세의 과세기간은 1월 1일부터 12월 31일까지 1년으로 한다.\n\n② 거주자가 사망한 경우

In [45]:
from langchain.chains import RetrievalQA


qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = retriever,
    chain_type_kwargs = {"prompt" : prompt}
)

# query -> 직장인 -> 거주자 chain 추가
ai_message = qa_chain({"query" : query})


In [46]:
ai_message

{'query': '연봉 5천만원인 거주자의 종합소득세는 얼마인가요?',
 'result': '연봉 5천만원인 거주자의 종합소득세를 계산하기 위한 구체적인 정보는 제공된 문서에서 확인할 수 없습니다. 종합소득세는 여러 공제 항목과 세율을 고려하여 복잡하게 계산되므로, 정확한 금액을 알기 위해서는 관련 세법과 공제 항목을 상세히 검토해야 합니다. 세무 전문가와 상담하는 것이 좋습니다.'}

In [48]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요
    사전 : {dictionary}
    질문 : {{question}}
    """
)

dictionary_chain = prompt | llm | StrOutputParser()

In [35]:
new_question = dictionary_chain.invoke({"question" : query})

In [36]:
tax_chain = {"query" : dictionary_chain} | qa_chain

In [37]:
ai_response = tax_chain.invoke({"question": query})

In [49]:
new_question

'사용자의 질문은 변경할 필요가 없습니다. \n\n변경된 질문: 연봉 5천만원인 거주자의 소득세는 얼마인가요?'

In [38]:
ai_response

{'query': '질문 : 연봉 5천만원인 거주자의 소득세는 얼마인가요?',
 'result': '해당 질문에 대한 답을 제공할 수 있는 맥락 정보가 없습니다. 연봉 5천만원에 대한 소득세 계산 방법이나 세율이 자료에 포함되어 있지 않습니다. 자세한 소득세 계산은 국세청 또는 세무 전문가에게 문의하는 것이 좋습니다.'}